In [ ]:
import os
import sys

sys.path.insert(0, os.path.abspath("../utils"))
from aoc_utils import load_data, check

In [ ]:
from collections import abc

In [ ]:
data = load_data(2023, 10)

In [ ]:
# data, part_1, part_2
tests = [
    (
        """.....
.S-7.
.|.|.
.L-J.
.....""",
        4,
        None,
    ),
    (
        """..F7.
.FJ|.
SJ.L7
|F--J
LJ...""",
        8,
        None,
    ),
    (
        """7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ""",
        8,
        None,
    ),
    (
        """...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........""",
        None,
        4,
    ),
    (
        """.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...""",
        None,
        8,
    ),
    (
        """FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L""",
        None,
        10,
    ),
]

# Part 1

In [ ]:
def get_maze(data):
    maze = data.splitlines()
    walled_maze = []
    for line in maze:
        walled_maze.append("." + line + ".")
    walled_maze = ["." * len(walled_maze[0])] + walled_maze + ["." * len(walled_maze[0])]
    maze_dct = {}
    for y, line in enumerate(walled_maze):
        for x, c in enumerate(line):
            maze_dct[x + 1j * y] = c
    return maze_dct

In [ ]:
def find_start(maze):
    for k in maze:
        if maze[k] == "S":
            return k

In [ ]:
# north: -i, south: i, east: 1, west: -1
pipes = {
    "|": {-1j, 1j},
    "-": {1, -1},
    "L": {-1j, 1},
    "J": {-1j, -1},
    "7": {1j, -1},
    "F": {1j, 1},
    ".": set(),
    "S": {-1j, 1j, 1, -1},
}
def follow_pipe(pipe, direction):
    if -direction not in pipes[pipe]:
        # end of the path
        return None
    d1, d2 = pipes[pipe]
    if d1 == -direction:
        return d2
    return d1

In [ ]:
def find_path(maze, start, direction, end_direction=False):
    pos = start
    path = {}
    length = 0
    while pos not in path:
        path[pos] = length
        pos += direction
        length += 1
        if maze[pos] == "S":
            if end_direction:
                return path, direction
            return path
        direction = follow_pipe(maze[pos], direction)
        if direction is None:
            return None
    assert False

In [ ]:
def compute_length(maze):
    lengths = []
    start = find_start(maze)
    for direction in (-1j, 1j, 1, -1):
        path = find_path(maze, start, direction)
        if path is not None:
            # instructions state that only one path is valid
            return (max(path.values()) + 1) // 2

In [ ]:
def get_steps(data):
    maze = get_maze(data)
    return compute_length(maze)

In [ ]:
check(get_steps, tests)
get_steps(data)

# Part 2

In [ ]:
def cleanup(maze):
    start = find_start(maze)
    for direction in (-1j, 1j, 1, -1):
        res = find_path(maze, start, direction, end_direction=True)
        if res is not None:
            path, s_dir = res
            S_dirs = {-s_dir, direction}
            break
    clean_maze = maze.copy()
    # find pipe type for S
    for type in pipes:
        if pipes[type] == S_dirs:
            clean_maze[start] = type
            break
    # remove unused pipes
    for cell in clean_maze:
        if cell not in path:
            clean_maze[cell] = "."
    return clean_maze

In [ ]:
def count_enclosed(data):
    maze = cleanup(get_maze(data))
    max_x = int(max(z.real for z in maze))
    max_y = int(max(z.imag for z in maze))
    cnt = 0
    for y in range(max_y + 1):
        enclosed = False
        for x in range(max_x + 1):
            c = maze[x + 1j * y]
            if c in "|F7":
                enclosed = not enclosed
            if c == "." and enclosed:
                cnt += 1
    return cnt

In [ ]:
check(count_enclosed, tests, 2)
count_enclosed(data)